In [ ]:
#Import and install all required packages and libraries

import urllib.request
import shutil
import os
import csv
import requests
import xml.etree.ElementTree as ET
import re

import pandas as pd
import sys
import os
from re import split
from time import strftime, sleep
from copy import copy

!pip3 install httpx
! pip install openai==0.28

import httpx
import time
import openai

import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
#EDIT ACCORDING TO REQUIREMENTS

#paper_doi='10.1016/j.jallcom.2011.12.116'
paper_doi='10.1016/j.jallcom.2011.12.116'

my_api_key=""
my_inst_token=""

openai.api_key = ""

In [ ]:
########## Object Retrieval API (retrieves XML of article using DOI)
def scopus_paper(paper_doi,apikey,insttoken):
    apikey=apikey
    headers={
        "X-ELS-APIKey":apikey,
        'X-ELS-Insttoken': insttoken,
        "Accept":'text/xml'
         }

    timeout = httpx.Timeout(10.0, connect=60.0)
    client = httpx.Client(timeout=timeout,headers=headers)
    query="&view=FULL"
    url= f"https://api.elsevier.com/content/article/doi/" + paper_doi
    r=client.get(url)
    print(r)
    #return r.text
    return r


########## Assistant Functions to Parse Article XML Format
def parse_title(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespaces
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}  # Update the ce namespace URI
    ns_dc = {'dc': 'http://purl.org/dc/elements/1.1/'}  # Update the dc namespace URI

    # Extract title
    title_element = root.find('.//dc:title', namespaces=ns_dc)
    if title_element is not None:
        title_element = title_element.text.strip()

    return title_element

def parse_description(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespaces
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}  # Update the ce namespace URI
    ns_dc = {'dc': 'http://purl.org/dc/elements/1.1/'}  # Update the dc namespace URI

    # Extract title
    title_element = root.find('.//dc:description', namespaces=ns_dc)
    if title_element is not None:
        title_element = title_element.text.strip()

    return title_element

def extract_para_ids(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespace
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}

    # Extract table IDs
    para_ids = [para.get('id') for para in root.findall('.//ce:para', namespaces=ns_ce)]

    return para_ids

def extract_para_by_id(xml_file_path, para_id):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespace
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}  # Update the ce namespace URI

    # Find the table element with the specified ID
    para_element = root.find(f'.//ce:para[@id="{para_id}"]', namespaces=ns_ce)

    if para_element is not None:
        # Convert the table element to an XML string
        para_xml = ET.tostring(para_element, encoding='unicode')
        return para_xml
    else:
        return None

def extract_table_ids(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespace
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}

    # Extract table IDs
    table_ids = [table.get('id') for table in root.findall('.//ce:table', namespaces=ns_ce)]

    return table_ids

def extract_table_by_id(xml_file_path, table_id):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Extract the namespace
    ns_ce = {'ce': 'http://www.elsevier.com/xml/common/dtd'}  # Update the ce namespace URI

    # Find the table element with the specified ID
    table_element = root.find(f'.//ce:table[@id="{table_id}"]', namespaces=ns_ce)

    if table_element is not None:
        # Convert the table element to an XML string
        table_xml = ET.tostring(table_element, encoding='unicode')
        return table_xml
    else:
        return None


########## Parse Article XML Format (to create TABLE CHUNKS & TEXT CHUNKS CSV Files)
def article_parser(paper_doi,my_api_key,my_inst_token):
  y = scopus_paper(paper_doi,my_api_key,my_inst_token)
  # saving the xml file
  with open('article.xml', 'wb') as f:
        f.write(y.content)
  xml_file_path = "article.xml"
  parsed_title = parse_title(xml_file_path)
  parsed_description = parse_description(xml_file_path)
  parsed_list_of_paras_xml=[]
  parsed_list_of_tables_xml=[]
  para_ids = extract_para_ids(xml_file_path)
  table_ids = extract_table_ids(xml_file_path)

  for para_id in para_ids:
        para_xml = extract_para_by_id(xml_file_path, para_id)
        parsed_list_of_paras_xml.append(para_xml)
  for table_id in table_ids:
        table_xml = extract_table_by_id(xml_file_path, table_id)
        parsed_list_of_tables_xml.append(table_xml)

  text_chunk_df = pd.DataFrame(parsed_list_of_paras_xml, columns=['passage'])
  text_chunk_df['passage'] = parsed_title + text_chunk_df['passage'].astype(str)
  text_chunk_df['doi']= paper_doi
  text_chunk_df.to_csv('text_chunk_df.csv')
  table_chunk_df = pd.DataFrame(parsed_list_of_tables_xml, columns=['passage'])
  table_chunk_df['doi']= paper_doi
  table_chunk_df.to_csv('table_chunk_df.csv')

  return None

########## OpenAI General Helper Code
dtime = strftime("%Y_%m_%d-%H%M%S")
START = 0
for i, arg in enumerate(sys.argv[1:]):
  if i==0:
    CSV_INPUT = sys.argv[i+1]
  if i==1:
    PROPERTY =  sys.argv[i+1]

dtime = dtime + "_" + CSV_INPUT

def prompt(Q,typ):
  if typ == 'yn':
    tkn = 6
  elif typ == 'all':
    tkn = 500
  elif typ == 'tab':
    tkn = 500
  while True:
    try:
      response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=Q,
        temperature=0,
        max_tokens=tkn,
        frequency_penalty=0,
        presence_penalty=0
      )
      break
    except Exception as e:
      print("An error occurred:", e)
      if 'Please reduce the length of the messages' in str(e):
        print('TRUNCATING')
        if 'Use only data present in the text. If data is not present in the text, type' in Q[1]["content"]:
          print(Q.pop(3))
          print(Q.pop(3))
        else:
          print(Q.pop(1))
          print(Q.pop(1))
      elif 'per min' in str(e):
          print("Sleeping for 15 sec.")
          sleep(15)
  return(Q,response['choices'][0]['message']['content'])

########## Generic Extraction Requirement Specific Code
PROPERTY= 'martensitic start temperature (Ms)'
PROPERTIES_TO_EXTRACT=['martensitic start temperature (Ms)','martensitic end temperature (Mf)','austenitic start temperature (As)','austenitic end temperature (Af)']

composition_format='[Ag(at%) Al(at%) Au(at%) B(at%)	Bi(at%)	Ce(at%)	Co(at%)	Cr(at%)	Cu(at%)	Dy(at%)	Er(at%)	Fe(at%)	Gd(at%)	Hf(at%)	In(at%)	La(at%)	Mn(at%)	Mo(at%)	Nb(at%)	Nd(at%)	Ni(at%)	Pb(at%)	Pd(at%)	Pr(at%)	Pt(at%)	Re(at%)	Rh(at%)	Sb(at%)	Sc(at%)	Si(at%)	Sn(at%)	Ta(at%)	Te(at%)	Ti(at%)	Tl(at%)	V(at%)	W(at%)	Y(at%)	Zr(at%)]'

########## Extraction Requirement Specific Code for TEXT CHUNKS
classif_q = 'Answer "Yes" or "No" only. Does the following XML text contain a value of '+PROPERTY+'?\n\n'

ifmulti_q = 'Answer "Yes" or "No" only. Does the following XML text contain more than one value of '+PROPERTY+'?\n\n'

single_q = [
'Give the number in unit of kelvin only, do not use a full sentence. If the value is not present in the text, type "None". What is the value of the '+PROPERTY+' in the following XML text?\n\n',
'Give the name of the material only, do not use a full sentence. If the name of the material is not present in the text, type "None". What is the material for which the '+PROPERTY+' is given in the following XML text?\n\n',
'Give only the answer value in number array format of: '+composition_format+' , do not use a full sentence. If value is not present in the text, type "None". What is the value of the composition for which the '+PROPERTY+' is given in the following XML text?\n\n',
'Give the number in unit of bar only, do not use a full sentence. If value is not present in the text, type "None". What is the value of the pressure for which the '+PROPERTY+' is given in the following XML text?\n\n',
'Give the value in format [x y z] only, do not use a full sentence. If value is not present in the text, type "None". What is the value of the orientation for which the '+PROPERTY+' is given in the following XML text?\n\n',
'Give the number in unit of kelvin only, do not use a full sentence. If value is not present in the text, type "None". What is the value of the temperature of aging for which the '+PROPERTY+' is given in the following XML text?\n\n',
'Give the number in unit of hours only, do not use a full sentence. If value is not present in the text, type "None". What is the value of the time duration of aging for which the '+PROPERTY+' is given in the following XML text?\n\n'
]

singlefollowup_q = [
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the '+PROPERTY+' in unit of kelvin for the compound/composition in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is "','" the compound for which the value of '+PROPERTY+' is given in the following XML text? Make sure it is a real compound.\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the composition in number array format of: '+composition_format+' for which the value of '+PROPERTY+' is given in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the pressure in unit of bar for which the value of '+PROPERTY+' is given in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the orientation in format [x y z] for which the value of '+PROPERTY+' is given in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the temperature of aging in unit of kelvin for which the value of '+PROPERTY+' is given in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the time duration of aging in unit of hour for which the value of '+PROPERTY+' is given in the following XML text?\n\n']
]

tab_q = 'In the table: For the value of the '+PROPERTY+', give the number in unit of kelvin only. For the value of the material, give the name of the material only. For the value of the composition, give the value in number array format of: '+composition_format+' only. For the value of the pressure, give the number in unit of bar only. For the value of the orientation, give the value in format [x y z] only. For the value of the temperature of aging, give the number in unit of kelvin only. For the value of the time duration of aging, give the number in unit of hours only.\n Use only data present in the XML text. If data is not present in the text, type "None". Summarize the values of '+PROPERTY+' in unit of kelvin in the following XML text in a form of a table consisting of: '+PROPERTY+' in unit of kelvin, material, composition, pressure in unit of bar, orientation in format [x y z], temperature of aging in unit of kelvin, time duration of aging in unit of hour.\n\n'

tabfollowup_q = [
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the '+PROPERTY+' in unit of kelvin for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the compound for which the value of the '+PROPERTY+' is ',' for the ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the following XML text? Make sure it is a real compound.\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the composition in number array format of: '+composition_format+' for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the pressure in unit of bar for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the orientation in format [x y z] for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the temperature of aging in unit of kelvin for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' time duration of aging in unit of hour in the following XML text?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the time duration of aging in unit of hour for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin in the following XML text?\n\n']
]

col = [PROPERTY+' in unit of kelvin','material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour']

single_cols = [PROPERTY+ ' in unit of kelvin','material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour']

########## Extraction Requirement Specific Code for TABLE CHUNKS
table_classif_q = 'Answer "Yes" or "No" only. Does the table contain a value of '+PROPERTY+'?\n\n'

table_q = 'In the table: For the value of the '+PROPERTY+', give the number in unit of kelvin only. For the value of the material, give the name of the material only. For the value of the composition, give the value in number array format of: '+composition_format+' only. For the value of the pressure, give the number in unit of bar only. For the value of the orientation, give the value in format [x y z] only. For the value of the temperature of aging, give the number in unit of kelvin only. For the value of the time duration of aging, give the number in unit of hours only.\n Use only data present in the XML text. If data is not present in the XML text, type "None". Summarize the values of '+PROPERTY+' in unit of kelvin in the following XML table in a form of a table consisting of: '+PROPERTY+' in unit of kelvin, material, composition, pressure in unit of bar, orientation in format [x y z], temperature of aging in unit of kelvin, time duration of aging in unit of hour.\n\n'

table_followup_q = [
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the '+PROPERTY+' in unit of kelvin for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the table?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the compound for which the value of the '+PROPERTY+' is ',' for the ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the table? Make sure it is a real compound.\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the composition in number array format of: '+composition_format+' for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the table?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the pressure in unit of bar for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the able?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the orientation in format [x y z] for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' temperature of aging in unit of kelvin, ',' time duration of aging in unit of hour in the table?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the temperature of aging in unit of kelvin for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' time duration of aging in unit of hour in the table?\n\n'],
['There is a possibility that the data you extracted is incorrect. Answer "Yes" or "No" only. Be very strict. Is ',' the value of the time duration of aging in unit of hour for which the value of the '+PROPERTY+' is ',' for the ',' compound, ',' composition in number array format of: '+composition_format+', ',' pressure in unit of bar, ',' orientation in format [x y z], ',' temperature of aging in unit of kelvin in the table?\n\n']
]

table_col = [PROPERTY+' in unit of kelvin','material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour']


########## Extract Data from TEXT CHUNKS CSV file using GPT
def extract_from_text_chunks():
 test_df = pd.read_csv("text_chunk_df.csv",encoding='unicode_escape')
 ntot=len(test_df)
 CSV_INPUT="text_chunk_df.csv"
 for i in range(START,len(test_df)):
  try:
    binary_classif=[]
    answers = []
    sss=[{"role": "system", "content": ""}]
    print("Processing ",CSV_INPUT," ",i," ",round(i/ntot*100,1),"%")
    ss = classif_q+test_df["passage"][i] #can remove the feature data "sentence" and keep only "passage"
    sss.append({"role": "user", "content": ss})
    sss,ans = prompt(sss,'yn')
    sss.append({"role": "assistant", "content": ans})

    if 'yes' in ans.strip().lower(): #if there is any value of PROPERTY in the text
      binary_classif.append(1)
      result = {}
      ss = ifmulti_q+test_df["passage"][i]
      sss.append({"role": "user", "content": ss})
      sss,ans = prompt(sss,'yn')
      sss.append({"role": "assistant", "content": ans})

      if 'no' in ans.lower(): #there is only one value of PROPERTY in the text
        print("single_value")
        result["passage"] = [test_df["passage"][i]]
        result["doi"] = [test_df["doi"][i]]
        result[PROPERTY+ " in unit of kelvin"] =[]
        result["material"] =[]
        result["composition in composition_format"] =[]
        result["pressure in unit of bar"] =[]
        result["orientation in format [x y z]"] =[]
        result["temperature of aging in unit of kelvin"] =[]
        result["time duration of aging in unit of hour"] =[]
        result[PROPERTY+" in unit of kelvin"+"_valid"] =[]
        result["material_valid"] =[]
        result["composition in composition_format_valid"] =[]
        result["pressure in unit of bar_valid"] =[]
        result["orientation in format [x y z]_valid"] =[]
        result["temperature of aging in unit of kelvin_valid"] =[]
        result["time duration of aging in unit of hour_valid"] =[]
        for j in range(len(single_q)):
          ss = single_q[j]+test_df["passage"][i]
          sss.append({"role": "user", "content": ss})
          sss,ans = prompt(sss,'all')
          sss.append({"role": "assistant", "content": ans})
          result[single_cols[j]].append(ans)
          if 'none' in ans.lower():
            result[single_cols[j]+"_valid"].append(0)
          else:
            result[single_cols[j]+"_valid"].append(1)

      elif 'yes' in ans.lower(): #there are more than one value of PROPERTY in the text
        print("multi_value")
        ss = tab_q+test_df["passage"][i]
        sss.append({"role": "user", "content": ss})
        sss,tab = prompt(sss,'tab') #tab is the tabulation made by GPT for text having multiple values of PROPERTY
        sss.append({"role": "assistant", "content": tab})
        sst = copy(sss)
        tab = [split('[,|]',row) for row in tab.strip().split('\n')]
        tab = [[item.strip() for item in row if len(item.strip())>0] for row in tab if len(row)>=6]
        if len(tab)<=0:
          tab.append([PROPERTY+ " in unit of kelvin",'material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour'])
        if len(tab)<=1:
          tab.append(['None','None','None','None','None','None','None'])
        else:
          tab.pop(1) #It is to remove the horizontal line below header

        tab.pop(0)
        head = [PROPERTY+ " in unit of kelvin",'material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour']
        tab = pd.DataFrame(tab,columns=head)
        result["passage"] = []
        result["doi"] = []
        result[PROPERTY+ " in unit of kelvin"] =[]
        result["material"] =[]
        result["composition in composition_format"] =[]
        result["pressure in unit of bar"] =[]
        result["orientation in format [x y z]"] =[]
        result["temperature of aging in unit of kelvin"] =[]
        result["time duration of aging in unit of hour"] =[]
        result[PROPERTY+" in unit of kelvin"+"_valid"] =[]
        result["material_valid"] =[]
        result["composition in composition_format_valid"] =[]
        result["pressure in unit of bar_valid"] =[]
        result["orientation in format [x y z]_valid"] =[]
        result["temperature of aging in unit of kelvin_valid"] =[]
        result["time duration of aging in unit of hour_valid"] =[]

        for k in range(len(tab)): #k is the counter/index for every row in tab
          sst.append({"role": "tab", "content": tab[col[0]][k]+","+tab[col[1]][k]+","+tab[col[2]][k]+","+tab[col[3]][k]+","+tab[col[4]][k]+","+tab[col[5]][k]})
          result["passage"].append(test_df["passage"][i])
          result["doi"].append(test_df["doi"][i])
          multi_valid = True

          ss1= tabfollowup_q[0][0]+str(tab[col[0]][k])+tabfollowup_q[0][1]+str(tab[col[1]][k])+tabfollowup_q[0][2]+str(tab[col[2]][k])+tabfollowup_q[0][3]+str(tab[col[3]][k])+tabfollowup_q[0][4]+str(tab[col[4]][k])+tabfollowup_q[0][5]+str(tab[col[5]][k])+tabfollowup_q[0][6]+str(tab[col[6]][k])+tabfollowup_q[0][7]+test_df["passage"][i]
          ss2= tabfollowup_q[1][0]+str(tab[col[1]][k])+tabfollowup_q[1][1]+str(tab[col[0]][k])+tabfollowup_q[1][2]+str(tab[col[2]][k])+tabfollowup_q[1][3]+str(tab[col[3]][k])+tabfollowup_q[1][4]+str(tab[col[4]][k])+tabfollowup_q[1][5]+str(tab[col[5]][k])+tabfollowup_q[1][6]+str(tab[col[6]][k])+tabfollowup_q[1][7]+test_df["passage"][i]
          ss3= tabfollowup_q[2][0]+str(tab[col[2]][k])+tabfollowup_q[2][1]+str(tab[col[0]][k])+tabfollowup_q[2][2]+str(tab[col[1]][k])+tabfollowup_q[2][3]+str(tab[col[3]][k])+tabfollowup_q[2][4]+str(tab[col[4]][k])+tabfollowup_q[2][5]+str(tab[col[5]][k])+tabfollowup_q[2][6]+str(tab[col[6]][k])+tabfollowup_q[2][7]+test_df["passage"][i]
          ss4= tabfollowup_q[3][0]+str(tab[col[3]][k])+tabfollowup_q[3][1]+str(tab[col[0]][k])+tabfollowup_q[3][2]+str(tab[col[1]][k])+tabfollowup_q[3][3]+str(tab[col[2]][k])+tabfollowup_q[3][4]+str(tab[col[4]][k])+tabfollowup_q[3][5]+str(tab[col[5]][k])+tabfollowup_q[3][6]+str(tab[col[6]][k])+tabfollowup_q[3][7]+test_df["passage"][i]
          ss5= tabfollowup_q[4][0]+str(tab[col[4]][k])+tabfollowup_q[4][1]+str(tab[col[0]][k])+tabfollowup_q[4][2]+str(tab[col[1]][k])+tabfollowup_q[4][3]+str(tab[col[2]][k])+tabfollowup_q[4][4]+str(tab[col[3]][k])+tabfollowup_q[4][5]+str(tab[col[5]][k])+tabfollowup_q[4][6]+str(tab[col[6]][k])+tabfollowup_q[4][7]+test_df["passage"][i]
          ss6= tabfollowup_q[5][0]+str(tab[col[5]][k])+tabfollowup_q[5][1]+str(tab[col[0]][k])+tabfollowup_q[5][2]+str(tab[col[1]][k])+tabfollowup_q[5][3]+str(tab[col[2]][k])+tabfollowup_q[5][4]+str(tab[col[3]][k])+tabfollowup_q[5][5]+str(tab[col[4]][k])+tabfollowup_q[5][6]+str(tab[col[6]][k])+tabfollowup_q[5][7]+test_df["passage"][i]
          ss7= tabfollowup_q[6][0]+str(tab[col[6]][k])+tabfollowup_q[6][1]+str(tab[col[0]][k])+tabfollowup_q[6][2]+str(tab[col[1]][k])+tabfollowup_q[6][3]+str(tab[col[2]][k])+tabfollowup_q[6][4]+str(tab[col[3]][k])+tabfollowup_q[6][5]+str(tab[col[4]][k])+tabfollowup_q[6][6]+str(tab[col[5]][k])+tabfollowup_q[6][7]+test_df["passage"][i]
          list_ss=[ss1,ss2,ss3,ss4,ss5,ss6,ss7]

          for l in range(7): #l is the counter/index for every column/feature to be extracted
            ss= list_ss[l]
            result[col[l]].append(tab[col[l]][k])
            if 'none' in tab[col[l]][k].lower():
              result[col[l]+"_valid"].append(0)
              multi_valid = False
            elif multi_valid:
              sss.append({"role": "user", "content": ss})
              sst.append({"role": "user", "content": ss})
              sss,ans = prompt(sss,'yn')
              sss.append({"role": "assistant", "content": ans})
              sst.append({"role": "assistant", "content": ans})
              if 'no' in ans.lower():
                result[col[l]+"_valid"].append(0)
                multi_valid = False
              else:
                result[col[l]+"_valid"].append(1)
            else:
              result[col[l]+"_valid"].append(1)

      try:
        if i==0:
          pd.DataFrame(result).to_csv("extracted_1.csv", mode='a', index=False, header=True)
        else:
          pd.DataFrame(result).to_csv("extracted_1.csv", mode='a', index=False, header=False)
      except Exception as e:
        print('Appending extracted gone wrong: ',i,"  ",e)
        print('Appending extracted gone wrong: ',result,"  ",e)
        print('Appending extracted gone wrong: ',tab,"  ",e)

    else: #no value of PROPERTY in the text
      print("no value of PROPERTY in the text")
      binary_classif.append(0)

    pd.DataFrame(binary_classif).to_csv("binclas_1", mode='a', index=False, header=False)

    try:
      pd.DataFrame(sst).to_csv("results_1", mode='a', index=False, header=False)
      del sst
    except:
      pd.DataFrame(sss).to_csv("results_1", mode='a', index=False, header=False)
  except Exception as e:
    print("GENERAL ERROR: ",e, ". Thus ignoring and proceeding to next line.")
 return None


########## Extract Data from TABLE CHUNKS CSV file using GPT
def extract_from_table_chunks():
 test_df = pd.read_csv("table_chunk_df.csv",encoding='unicode_escape')
 ntot=len(test_df)
 CSV_INPUT="table_chunk_df.csv"
 for i in range(START,len(test_df)):
  try:
    binary_classif=[]
    answers = []
    sss=[{"role": "system", "content": ""}]
    print("Processing ",CSV_INPUT," ",i," ",round(i/ntot*100,1),"%")
    ss = table_classif_q+test_df["passage"][i] #can remove the feature data "sentence" and keep only "passage"
    sss.append({"role": "user", "content": ss})
    sss,ans = prompt(sss,'yn')
    sss.append({"role": "assistant", "content": ans})

    #if 'yes' in ans.strip().lower(): #if there is any value of PROPERTY in the text
    x='yes'
    if x=='yes':
      binary_classif.append(1)
      result = {}

      table='yes'
      if table=='yes':
        print("multi_value")
        ss = table_q+test_df["passage"][i]
        sss.append({"role": "user", "content": ss})
        sss,tab = prompt(sss,'tab') #tab is the tabulation made by GPT for text having multiple values of PROPERTY
        sss.append({"role": "assistant", "content": tab})
        sst = copy(sss)
        tab = [split('[,|]',row) for row in tab.strip().split('\n')]
        tab = [[item.strip() for item in row if len(item.strip())>0] for row in tab if len(row)>=6]
        if len(tab)<=0:
          tab.append([PROPERTY+ " in unit of kelvin",'material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour'])
        if len(tab)<=1:
          tab.append(['None','None','None','None','None','None','None'])
        else:
          tab.pop(1) #It is to remove the horizontal line below header

        tab.pop(0)
        head = [PROPERTY+ " in unit of kelvin",'material','composition in composition_format','pressure in unit of bar','orientation in format [x y z]','temperature of aging in unit of kelvin','time duration of aging in unit of hour']
        tab = pd.DataFrame(tab,columns=head)
        result["passage"] = []
        result["doi"] = []
        result[PROPERTY+ " in unit of kelvin"] =[]
        result["material"] =[]
        result["composition in composition_format"] =[]
        result["pressure in unit of bar"] =[]
        result["orientation in format [x y z]"] =[]
        result["temperature of aging in unit of kelvin"] =[]
        result["time duration of aging in unit of hour"] =[]
        result[PROPERTY+" in unit of kelvin"+"_valid"] =[]
        result["material_valid"] =[]
        result["composition in composition_format_valid"] =[]
        result["pressure in unit of bar_valid"] =[]
        result["orientation in format [x y z]_valid"] =[]
        result["temperature of aging in unit of kelvin_valid"] =[]
        result["time duration of aging in unit of hour_valid"] =[]

        for k in range(len(tab)): #k is the counter/index for every row in tab
          sst.append({"role": "tab", "content": tab[table_col[0]][k]+","+tab[table_col[1]][k]+","+tab[table_col[2]][k]+","+tab[table_col[3]][k]+","+tab[table_col[4]][k]+","+tab[table_col[5]][k]})
          result["passage"].append(test_df["passage"][i])
          result["doi"].append(test_df["doi"][i])
          multi_valid = True

          ss1= table_followup_q[0][0]+str(tab[table_col[0]][k])+table_followup_q[0][1]+str(tab[table_col[1]][k])+table_followup_q[0][2]+str(tab[table_col[2]][k])+table_followup_q[0][3]+str(tab[table_col[3]][k])+table_followup_q[0][4]+str(tab[table_col[4]][k])+table_followup_q[0][5]+str(tab[table_col[5]][k])+table_followup_q[0][6]+str(tab[table_col[6]][k])+table_followup_q[0][7]+test_df["passage"][i]
          ss2= table_followup_q[1][0]+str(tab[table_col[1]][k])+table_followup_q[1][1]+str(tab[table_col[0]][k])+table_followup_q[1][2]+str(tab[table_col[2]][k])+table_followup_q[1][3]+str(tab[table_col[3]][k])+table_followup_q[1][4]+str(tab[table_col[4]][k])+table_followup_q[1][5]+str(tab[table_col[5]][k])+table_followup_q[1][6]+str(tab[table_col[6]][k])+table_followup_q[1][7]+test_df["passage"][i]
          ss3= table_followup_q[2][0]+str(tab[table_col[2]][k])+table_followup_q[2][1]+str(tab[table_col[0]][k])+table_followup_q[2][2]+str(tab[table_col[1]][k])+table_followup_q[2][3]+str(tab[table_col[3]][k])+table_followup_q[2][4]+str(tab[table_col[4]][k])+table_followup_q[2][5]+str(tab[table_col[5]][k])+table_followup_q[2][6]+str(tab[table_col[6]][k])+table_followup_q[2][7]+test_df["passage"][i]
          ss4= table_followup_q[3][0]+str(tab[table_col[3]][k])+table_followup_q[3][1]+str(tab[table_col[0]][k])+table_followup_q[3][2]+str(tab[table_col[1]][k])+table_followup_q[3][3]+str(tab[table_col[2]][k])+table_followup_q[3][4]+str(tab[table_col[4]][k])+table_followup_q[3][5]+str(tab[table_col[5]][k])+table_followup_q[3][6]+str(tab[table_col[6]][k])+table_followup_q[3][7]+test_df["passage"][i]
          ss5= table_followup_q[4][0]+str(tab[table_col[4]][k])+table_followup_q[4][1]+str(tab[table_col[0]][k])+table_followup_q[4][2]+str(tab[table_col[1]][k])+table_followup_q[4][3]+str(tab[table_col[2]][k])+table_followup_q[4][4]+str(tab[table_col[3]][k])+table_followup_q[4][5]+str(tab[table_col[5]][k])+table_followup_q[4][6]+str(tab[table_col[6]][k])+table_followup_q[4][7]+test_df["passage"][i]
          ss6= table_followup_q[5][0]+str(tab[table_col[5]][k])+table_followup_q[5][1]+str(tab[table_col[0]][k])+table_followup_q[5][2]+str(tab[table_col[1]][k])+table_followup_q[5][3]+str(tab[table_col[2]][k])+table_followup_q[5][4]+str(tab[table_col[3]][k])+table_followup_q[5][5]+str(tab[table_col[4]][k])+table_followup_q[5][6]+str(tab[table_col[6]][k])+table_followup_q[5][7]+test_df["passage"][i]
          ss7= table_followup_q[6][0]+str(tab[table_col[6]][k])+table_followup_q[6][1]+str(tab[table_col[0]][k])+table_followup_q[6][2]+str(tab[table_col[1]][k])+table_followup_q[6][3]+str(tab[table_col[2]][k])+table_followup_q[6][4]+str(tab[table_col[3]][k])+table_followup_q[6][5]+str(tab[table_col[4]][k])+table_followup_q[6][6]+str(tab[table_col[5]][k])+table_followup_q[6][7]+test_df["passage"][i]
          list_ss=[ss1,ss2,ss3,ss4,ss5,ss6,ss7]

          for l in range(7): #l is the counter/index for every column/feature to be extracted
            ss= list_ss[l]
            result[table_col[l]].append(tab[table_col[l]][k])
            if 'none' in tab[table_col[l]][k].lower():
              result[table_col[l]+"_valid"].append(0)
              multi_valid = False
            elif multi_valid:
              sss.append({"role": "user", "content": ss})
              sst.append({"role": "user", "content": ss})
              sss,ans = prompt(sss,'yn')
              sss.append({"role": "assistant", "content": ans})
              sst.append({"role": "assistant", "content": ans})
              if 'no' in ans.lower():
                result[table_col[l]+"_valid"].append(0)
                multi_valid = False
              else:
                result[table_col[l]+"_valid"].append(1)
            else:
              result[table_col[l]+"_valid"].append(1)

      try:
        if i==0:
          pd.DataFrame(result).to_csv("extracted_2.csv", mode='a', index=False, header=True)
        else:
          pd.DataFrame(result).to_csv("extracted_2.csv", mode='a', index=False, header=False)
      except Exception as e:
        print('Appending extracted gone wrong: ',i,"  ",e)
        print('Appending extracted gone wrong: ',result,"  ",e)
        print('Appending extracted gone wrong: ',tab,"  ",e)

    else: #no value of PROPERTY in the text
      print("no value of PROPERTY in the tables")
      binary_classif.append(0)

    pd.DataFrame(binary_classif).to_csv("binclas_2", mode='a', index=False, header=False)

    try:
      pd.DataFrame(sst).to_csv("results_2", mode='a', index=False, header=False)
      del sst
    except:
      pd.DataFrame(sss).to_csv("results_2", mode='a', index=False, header=False)
  except Exception as e:
    print("GENERAL ERROR: ",e, ". Thus ignoring and proceeding to next line.")
 return None


In [ ]:
def final_workflow(paper_doi,my_api_key,my_inst_token):
  parse_articles= article_parser(paper_doi,my_api_key,my_inst_token) #create TABLE CHUNKS & TEXT CHUNKS CSV Files for the given Article DOI
  extract_data_from_text_chunks= extract_from_text_chunks()
  extract_data_from_table_chunks= extract_from_table_chunks()
  return None

In [ ]:
final_workflow(paper_doi,my_api_key,my_inst_token)

<Response [200 OK]>
Processing  text_chunk_df.csv   0   0.0 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   1   5.9 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   2   11.8 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   3   17.6 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   4   23.5 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   5   29.4 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   6   35.3 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   7   41.2 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   8   47.1 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   9   52.9 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   10   58.8 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   11   64.7 %
no value of PROPERTY in the text
Processing  text_chunk_df.csv   12   70.6 %
no value of PROPERTY in 

In [ ]:
text_df = pd.read_csv("extracted_1.csv") #dataset extracted from paragraphs in the article
print(text_df.head()) #NOTE: "extracted_1.csv" will only exist if the paragraph/s in the article contained value of PROPERTY

FileNotFoundError: ignored

In [ ]:
tables_df = pd.read_csv("extracted_2.csv") #dataset extracted from tables in the article
print(tables_df.head()) #NOTE: "extracted_2.csv" will only exist if the table/s in the article contained value of PROPERTY